초짜 데이터 분석가의 R 
==========

기본적으로 jupyter에서 다룰 수 있는 라이브러리의 갯수는 한정되어 있다. 하지만 jupyter는 code에 대한 결과값을 바로 알 수 있을 뿐 아니라, 코드와 결과값을 연결하여 정리할 수 있다는 점에서 편리하다. 특히 시각화 라이브러리를 지원하고 있어서 코드 결과를 확인하는 데 유용하다. 현재까지 확인한 걸로는 sampling과 관련된 라이브러리가 거의 인식되지 않고, R에서 별도로 설치한 후 jupyter notebook을 실행해도 해당 라이브러리가 설치되지 않았다고 뜨는 등의 문제가 발생한다.

jupyter notebook에서 R을 활용하려면 아래의 방법 중 하나를 골라서 해보자. 물론 Rstudio를 사용해 아래의 코드들을 실행해도 관계없다.
* 터미널에서 anaconda를 통해 R을 설치한다 
* anaconda navigator라는 프로그램을 통해 RStudio를 설치한다 

경로 설정
-------

R을 설치했다면, 이제 데이터 분석을 위해 R을 시작해보자. 우선 getwd를 통해 디렉토리 경로를 확인하고 setwd()를 통해 경로를 확인하자. 
분석하려는 데이터가 들어있는 경로로 디렉토리를 설정해주면 된다. 

In [ ]:
getwd()

In [ ]:
setwd('/Users/Documents')

데이터 불러오기
-----------

R에서 데이터를 분석하기 위해서 가장 먼저 해야 할 일은 데이터를 불러오는 일이다. 데이터를 불러오는 과정에서 활용할 수 있는 함수는 데이터의 확장자에 따라 여러 라이브러리를 제공하고 있다. 
* read_excel : 윈도우에서만 지원
* read.table : 데이터의 컬럼 구분이 , 로 이뤄진 csv를 불러올 때 사용 
* read.csv : 기본적으로 csv 파일을 불러올 때 사용

In [ ]:
data = read.table("data.csv")
data = read.csv("data.csv")

In [ ]:
#libray(readexcel)
data = read_excel("data.xlsx")

위의 함수 중 데이터의 성격에 맞는 것을 사용하여 데이터를 불러온 후, 

데이터 편집을 위한 라이브러리는 기본적으로 아래의 것들이 사용된다. 요즘엔 data.table이 data.frame보다 빨라서 많이들 찾고 있다.
* data.table
* data.frame

데이터 가공
--------------------

데이터를 불러왔다면 데이터의 상태를 먼저 확인해야 한다. 아래의 함수를 활용해보자
* str()
* summary()

In [4]:
str(data)

In [ ]:
summary(data)

그 다음으로 해야할 것은 data의 결측치를 확인하고 결측치를 삭제하고 정리하는 작업을 진행해야 한다. 결측치를 확인하기 위해 아래의 함수들을 활용하여 
* is.na 
* na.omit 

In [ ]:
sum(is.na(data))
data <- na.omit(data)

데이터 탐색
---------

데이터를 정제했으면 다음 단계에서 필요한 것은 데이터 탐색이다. 아래의 기능들을 참고하여 
* 히스토그램
* 왜도, 첨도

In [6]:
hist(data$var1) #히스토그램 

In [ ]:
skewness(data$var1) #왜도

In [ ]:
kurtosis(data$var1) #첨도

샘플링
-----

히스토그램의 분포를 확인한 뒤, 일부 데이터를 추출하여 샘플링을 진행하면 된다. 
* 일반샘플링
* 층화샘플링 
* 오버샘플링 

불균형 표본의 문제를 해결하기 위한 데이터 추출법을 조사하기 위해 아래의 아티클을 참고했다. 
* dealing with imbalanced classification problems in R, https://www.analyticsvidhya.com/blog/2016/03/practical-guide-deal-imbalanced-classification-problems/


In [ ]:
sample(, replace = FALSE) #단순임의

In [ ]:
require("data.table")
require("sampling")
data2= strata(data, stratanames="REG", size=3000, method="srswor") #층화샘플링 

#"srswor" = sample random sampling without replacement, basic.
#strata(data, stratanames=NULL, size, method=c("srswor", "srswr", "poisson", "systematic"), pik,description=FALSE)

In [ ]:
#ROSE and DMwR
library("ROSE")
data2 <- ovun.sample(cls ~ ., data = data, method = "over", N = 3059) #oversampling

또 표준정규화를 진행하기 위해서는 아래의 함수를 사용하면 된다 
* scale()

In [ ]:
scale() #표준정규화 http://rfriend.tistory.com/52 #alternative: (x-mean(x))/sd(x)


data2 = scale(
                  data,            # 숫자 벡터 유형의 객체
                  center=TRUE,  # TRUE면 모든 데이터에서 전체 데이터의 평균을 뺀다.
                  # scale이 TRUE일 때 center도 TRUE면 모든 데이터를 전체 데이터의 표준 편차로 나눈다.
                  # scale이 TRUE지만 center는 FALSE면 모든 데이터를 전체 데이터의 제곱 평균 제곱근으로 나눈다.
                  # scale이 FALSE면 데이터를 어떤 값으로도 나누지 않는다.
                  scale=TRUE)


컬럼 가공 및 추출
------------- 


특정 데이터에서 컬럼을 추출하거나 제외하고, 컬럼의 순서를 바꿀 때는 아래의 기능들을 활용하면 된다. 
* data[c(n:m)] : data에서 c의 범위에 해당하는 컬럼만을 추출한다 
* data[-c(n:m)] : data에서 c의 범위에 해당하는 컬럼을 제외한다
* data[c(n, m, o, p, q, r, s)] : 컬럼의 순서를 교체하기 위해 사용한다 

In [7]:
#컬럼 가공 및 추출 
data2 = data[c(1:10)] 
data2 = data[-c(2:5)] 
data3 = data[c(n, m, o, p, q, r)]

조건을 만족하는 행을 추출하고 싶다면 subset에서 특정 컬럼에 조건을 걸어 데이터를 추출해볼 수 있다.
* subset(data, x>=100)


그리고 데이터에 컬럼을 추가해보고 싶다면 cbind를 이용해보자.
* cbind()

In [ ]:
subset() #조건을 만족하는 행 추출  
cbind() #컬럼 추가 

컬럼의 이름을 바꾸기 위해서는 아래의 명령어를 활용할 수 있다. 
* colnames(data) = c('name1', 'name2')

In [2]:
colnames(data2) <- c('name', 'value')

데이터 타입 바꾸기
-------------

str()을 통해서 데이터의 성격을 확인한 후 상황에 따라 본인이 원하는 타입으로 데이터의 성격을 바꿀 수 있다. 

* as.numeric
* as.character
* as.factor

In [ ]:
data$var1 = as.numeric(data$var1)
data$var1 = as.character(data$var1)
data$var1 = as.factor(data$var1)

데이터 목록을 data.frame, 열과 행의 구성으로 바꾸고 싶다면 as.data.frame을 활용하자.

In [ ]:
data$var1 = as.data.frame(data$var1)

조건에 맞는 행 추출하기 
-----------------

subset은 1가지 조건에 해당하는 열의 값들을 추출하여 새로운 행을 만드는 데 이용된다. 하지만 만약 2가지 이상의 카테고리를 포함한 데이터에서 조건에 맞는 행을 추출해야 한다면, 아래의 수식을 사용해보는 것도 좋다. 
* transform 


In [ ]:
data2 <- transform(data, channel = ifelse(variable1 == "one", 1,
                                   ifelse(variable1 == "two", 2, 0
                                         )))